In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nn_utils
import pandas as pd

In [2]:
from os import listdir
import datetime

In [3]:
!ls ../data/projekt4/train

humidity_train.csv  temperature_train.csv	   wind_direction_train.csv
pressure_train.csv  weather_description_train.csv  wind_speed_train.csv


In [4]:
train_dir = "../data/projekt4/train"
test_dir = "../data/projekt4/test"

In [5]:
df_city = pd.read_csv("../data/projekt4/city_attributes.csv", sep=';')

In [6]:
df_city

,City,Country,Latitude,Longitude
0,Vancouver,Canada,49.249660,-123.119339
1,Portland,United States,45.523449,-122.676208
2,San Francisco,United States,37.774929,-122.419418
3,Seattle,United States,47.606209,-122.332069
4,Los Angeles,United States,34.052231,-118.243683
5,San Diego,United States,32.715328,-117.157257
6,Las Vegas,United States,36.174969,-115.137222
7,Phoenix,United States,33.448380,-112.074043
8,Albuquerque,United States,35.084492,-106.651138
9,Denver,United States,39.739151,-104.984703


In [7]:
df_humidity = pd.read_csv("../data/projekt4/train/humidity_train.csv", sep=';')

In [8]:
df_dict = {}
for file in listdir(train_dir):
    name = file.split("_train")[0]
    print(name)
    df_dict[name] = pd.read_csv(train_dir+"/"+file, sep=';', parse_dates=['datetime'])

temperature
wind_speed
pressure
wind_direction
weather_description
humidity


In [9]:
df_by_city = dict()
for city in df_city['City']:
    df_by_city[city] = pd.DataFrame()
    #Datetime is the same across all tables nothing lacking or omited
    df_by_city[city]['datetime'] = df_dict['pressure']['datetime']
    for key in df_dict.keys():
        df_by_city[city][key] = df_dict[key][city]

In [10]:
pd.set_option('display.max_rows', 50)

In [11]:
df_portland = df_by_city['Portland']

In [12]:
def get_date_info(df, col_name = 'datetime'):
    df['date'] = df_portland['datetime'].apply(lambda x: x.date())
    #this is usefull since it carries quite a bit of info
    df['day'] = df_portland['datetime'].apply(lambda x: x.day_of_year)
    df['year'] = df_portland['datetime'].apply(lambda x: x.year)
    #df['hour'] = df_portland['datetime'].apply(lambda x: x.hour)

    return df
    

In [13]:
df_portland = get_date_info(df_portland)

In [14]:
transform = df_portland.columns.drop(['datetime','date', 'weather_description'])
agg_df = df_portland.groupby('date')[transform].mean()
agg_df = agg_df.reset_index()

In [15]:
date = datetime.date(2003,8,1)
for i in range(5): 
    date += datetime.timedelta(days=1)
    print(date)

2003-08-02
2003-08-03
2003-08-04
2003-08-05
2003-08-06


In [177]:
#ok so we need to do some data trickery in here to have it to work. 
agg_df.sort_values('date')

,date,temperature,wind_speed,pressure,wind_direction,humidity,day,year
0,2012-01-10,282.118197,0.000000,1024.000000,57.727273,78.727273,10.0,2012.0
1,2012-01-11,286.934167,3.166667,1007.041667,119.166667,92.909091,11.0,2012.0
2,2012-01-12,282.205417,5.625000,999.291667,130.416667,82.750000,12.0,2012.0
3,2012-02-10,286.137728,1.291667,1023.583333,214.041667,65.833333,41.0,2012.0
4,2012-02-11,285.686250,3.666667,1008.708333,153.333333,83.947368,42.0,2012.0
...,...,...,...,...,...,...,...,...
1517,2016-12-07,291.421762,2.333333,1016.166667,244.916667,61.083333,342.0,2016.0
1518,2016-12-08,297.536250,0.500000,1014.500000,250.291667,60.041667,343.0,2016.0
1519,2016-12-09,289.096667,2.708333,1019.708333,239.333333,60.250000,344.0,2016.0
1520,2016-12-10,281.114250,1.333333,996.541667,77.750000,67.250000,345.0,2016.0


In [113]:
df_x = pd.DataFrame()
df_y = pd.DataFrame()
df = agg_df
for record in df.iterrows():
    x_row = record[1]
    y_date = x_row.date
    next_date = x_row.date
    add_record = True
    for i in range(1,3):
        next_date += datetime.timedelta(days=1)
        try:
            next_day = df[df['date'] == next_date].iloc[0]
            next_day.index = [k+f"_{i}" for k in next_day.index]
            x_row = pd.concat([x_row, next_day])
        except IndexError:
            add_record = False
            break
    next_date += datetime.timedelta(days=1)
    try:
        y_date += datetime.timedelta(days=4)
        y_row = df[df['date'] == y_date].iloc[0]
    except IndexError:
        add_record = False
        continue
    if add_record:
        df_x = df_x.append(x_row, ignore_index=True)
        df_y = df_y.append(y_row, ignore_index=True)

In [114]:
df_x

,date,temperature,wind_speed,pressure,wind_direction,humidity,day,year,date_1,temperature_1,...,day_1,year_1,date_2,temperature_2,wind_speed_2,pressure_2,wind_direction_2,humidity_2,day_2,year_2
0,2012-10-10,284.363958,0.708333,1017.416667,176.041667,62.600000,284.0,2012.0,2012-10-11,277.946250,...,285.0,2012.0,2012-10-12,278.335417,1.833333,1028.458333,103.333333,90.291667,286.0,2012.0
1,2012-10-11,277.946250,0.708333,1011.083333,157.916667,84.333333,285.0,2012.0,2012-10-12,278.335417,...,286.0,2012.0,2012-10-13,284.839583,1.666667,1013.416667,156.125000,77.521739,287.0,2012.0
2,2012-10-12,278.335417,1.833333,1028.458333,103.333333,90.291667,286.0,2012.0,2012-10-13,284.839583,...,287.0,2012.0,2012-10-14,286.330833,3.000000,1020.625000,177.083333,88.416667,288.0,2012.0
3,2012-10-13,284.839583,1.666667,1013.416667,156.125000,77.521739,287.0,2012.0,2012-10-14,286.330833,...,288.0,2012.0,2012-10-15,287.928333,2.625000,1020.291667,144.166667,86.150000,289.0,2012.0
4,2012-10-14,286.330833,3.000000,1020.625000,177.083333,88.416667,288.0,2012.0,2012-10-15,287.928333,...,289.0,2012.0,2012-10-16,288.754583,3.125000,1014.000000,163.333333,86.041667,290.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,2016-12-03,281.283024,4.166667,1004.416667,168.791667,86.375000,338.0,2016.0,2016-12-04,285.519475,...,339.0,2016.0,2016-12-05,291.789167,1.583333,1015.208333,258.916667,39.375000,340.0,2016.0
1446,2016-12-04,285.519475,2.916667,1021.041667,188.041667,61.500000,339.0,2016.0,2016-12-05,291.789167,...,340.0,2016.0,2016-12-06,287.869583,0.625000,1023.000000,271.708333,77.000000,341.0,2016.0
1447,2016-12-05,291.789167,1.583333,1015.208333,258.916667,39.375000,340.0,2016.0,2016-12-06,287.869583,...,341.0,2016.0,2016-12-07,291.421762,2.333333,1016.166667,244.916667,61.083333,342.0,2016.0
1448,2016-12-06,287.869583,0.625000,1023.000000,271.708333,77.000000,341.0,2016.0,2016-12-07,291.421762,...,342.0,2016.0,2016-12-08,297.536250,0.500000,1014.500000,250.291667,60.041667,343.0,2016.0


In [115]:
df_x = df_x.drop(columns=[x for x in df_x.columns if 'date' in x])

In [116]:
df_x

,temperature,wind_speed,pressure,wind_direction,humidity,day,year,temperature_1,wind_speed_1,pressure_1,...,humidity_1,day_1,year_1,temperature_2,wind_speed_2,pressure_2,wind_direction_2,humidity_2,day_2,year_2
0,284.363958,0.708333,1017.416667,176.041667,62.600000,284.0,2012.0,277.946250,0.708333,1011.083333,...,84.333333,285.0,2012.0,278.335417,1.833333,1028.458333,103.333333,90.291667,286.0,2012.0
1,277.946250,0.708333,1011.083333,157.916667,84.333333,285.0,2012.0,278.335417,1.833333,1028.458333,...,90.291667,286.0,2012.0,284.839583,1.666667,1013.416667,156.125000,77.521739,287.0,2012.0
2,278.335417,1.833333,1028.458333,103.333333,90.291667,286.0,2012.0,284.839583,1.666667,1013.416667,...,77.521739,287.0,2012.0,286.330833,3.000000,1020.625000,177.083333,88.416667,288.0,2012.0
3,284.839583,1.666667,1013.416667,156.125000,77.521739,287.0,2012.0,286.330833,3.000000,1020.625000,...,88.416667,288.0,2012.0,287.928333,2.625000,1020.291667,144.166667,86.150000,289.0,2012.0
4,286.330833,3.000000,1020.625000,177.083333,88.416667,288.0,2012.0,287.928333,2.625000,1020.291667,...,86.150000,289.0,2012.0,288.754583,3.125000,1014.000000,163.333333,86.041667,290.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,281.283024,4.166667,1004.416667,168.791667,86.375000,338.0,2016.0,285.519475,2.916667,1021.041667,...,61.500000,339.0,2016.0,291.789167,1.583333,1015.208333,258.916667,39.375000,340.0,2016.0
1446,285.519475,2.916667,1021.041667,188.041667,61.500000,339.0,2016.0,291.789167,1.583333,1015.208333,...,39.375000,340.0,2016.0,287.869583,0.625000,1023.000000,271.708333,77.000000,341.0,2016.0
1447,291.789167,1.583333,1015.208333,258.916667,39.375000,340.0,2016.0,287.869583,0.625000,1023.000000,...,77.000000,341.0,2016.0,291.421762,2.333333,1016.166667,244.916667,61.083333,342.0,2016.0
1448,287.869583,0.625000,1023.000000,271.708333,77.000000,341.0,2016.0,291.421762,2.333333,1016.166667,...,61.083333,342.0,2016.0,297.536250,0.500000,1014.500000,250.291667,60.041667,343.0,2016.0


In [117]:
df_y = df_y['temperature']

In [129]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
scaler = MaxAbsScaler()
scaler.fit(df_x)
scaled = scaler.transform(df_x)
scaled_df = pd.DataFrame(scaled, columns=df_x.columns)

In [130]:
train_x = tf.convert_to_tensor(scaled_df[:1200], dtype=tf.float32)
train_y = tf.convert_to_tensor(df_y[:1200], dtype=tf.float32)
test_x = tf.convert_to_tensor(scaled_df[1200:], dtype=tf.float32)
test_y = tf.convert_to_tensor(df_y[1200:], dtype=tf.float32)


In [131]:
from neural_network import *

In [181]:
#Add random seet to network so it can be repeatable
net = NeuralNetworkNew([21,128,32,16,1], net_type='regression')
batch_size = 512
epochs = 100
steps_per_epoch = int(np.ceil(train_x.shape[0]/ batch_size))
lr = 1e-5

In [182]:
net.train(
    train_x, train_y,
    test_x, test_y,
    epochs, steps_per_epoch, 
    batch_size, lr
    
)

Epoch 0....val metric: 62601.652
Epoch 1....val metric: 24763.738
Epoch 2....val metric: 154.59236
Epoch 3....val metric: 43.303234
Epoch 4....val metric: 43.25088
Epoch 5....val metric: 43.214485
Epoch 6....val metric: 43.17837
Epoch 7....val metric: 43.142544
Epoch 8....val metric: 43.107136
Epoch 9....val metric: 43.07203
Epoch 10....val metric: 43.037506
Epoch 11....val metric: 43.003048
Epoch 12....val metric: 42.96867
Epoch 13....val metric: 42.935078
Epoch 14....val metric: 42.90181
Epoch 15....val metric: 42.86879
Epoch 16....val metric: 42.836143
Epoch 17....val metric: 42.80375
Epoch 18....val metric: 42.77168
Epoch 19....val metric: 42.7401
Epoch 20....val metric: 42.708767
Epoch 21....val metric: 42.677677
Epoch 22....val metric: 42.64682
Epoch 23....val metric: 42.61642
Epoch 24....val metric: 42.58628
Epoch 25....val metric: 42.556408
Epoch 26....val metric: 42.526745
Epoch 27....val metric: 42.497192
Epoch 28....val metric: 42.468296
Epoch 29....val metric: 42.439255
Epo

{'val_loss': 35.130474,
 'train_loss': 146.69903945922852,
 'val_metrics': [],
 'val_metric': 40.86163}

In [128]:
for i in zip(net.predict(test_x), test_y):
    print(i[0].numpy(),i[1].numpy())

[275.86642] 270.90433
[288.4086] 279.468
[274.97757] 284.01782
[271.154] 286.11905
[271.9478] 291.28915
[290.5309] 291.8871
[280.1144] 291.87958
[290.0162] 291.2725
[295.39102] 287.57874
[288.0839] 283.69458
[289.80106] 286.12543
[285.83603] 280.33163
[277.65903] 280.46902
[281.1681] 284.56302
[287.06674] 285.293
[278.22614] 285.56366
[290.32318] 283.2313
[290.28226] 281.43307
[276.68423] 282.3787
[274.53564] 282.40234
[282.50296] 281.456
[286.23642] 281.20068
[286.00473] 282.90616
[285.1426] 279.92914
[281.67862] 281.29794
[274.834] 284.50784
[271.6107] 286.65552
[283.52835] 272.65677
[290.23035] 281.41446
[268.68716] 282.8718
[269.43973] 285.6688
[266.11057] 287.20792
[294.88934] 298.48416
[273.86972] 289.5446
[285.59445] 295.05542
[295.38837] 287.65207
[291.5733] 285.03833
[291.3359] 284.5275
[287.174] 283.257
[286.70493] 288.7041
[284.1525] 293.45206
[292.38596] 293.1467
[295.34073] 291.83005
[295.25064] 290.16125
[284.95523] 286.7865
[275.34116] 285.68887
[277.22327] 284.4201
[278